In [27]:
import setup
setup.init_django()

In [28]:
from market.models import StockQuote

In [29]:
from django.db.models import Avg, F, RowRange, Window, Max, Min
from django.db.models.functions import TruncDate, FirstValue
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [30]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now
ticker = "AAPL"

In [31]:
latest_daily_timestamps = (
    StockQuote.timescale.filter(company__ticker = ticker, time__range=(start_date - timedelta(days=40), end_date))
    .time_bucket('time', '1 day')
    .annotate(date=TruncDate('time'))
    .values('company', 'date')
    .annotate(latest_time=Max('time'))
    .values('company', 'date', 'latest_time')
    .order_by('date') # skipping over dates not in database
)

# timestamps for final of the day
actual_timestamps = list(set([x['latest_time'] for x in latest_daily_timestamps]))
len(actual_timestamps)

53

In [32]:
qs = StockQuote.objects.filter(
    company__ticker = ticker, 
    time__range=(start_date, end_date),
    time__in=actual_timestamps
)

In [33]:
def get_volume_trend(queryset, days=5):
    # Analyze recent volume trends
    start = -(days-1)
    data = queryset.annotate(
        avg_volume=Window(
            expression = Avg('volume'), # getting average trading volume
            order_by=F('time').asc(), # orders by time
            partition_by=[],
            frame = RowRange(start=start, end=0) 
        )
    ).order_by('-time').first() # first is to grab most recent data

    if not data:
        return None

    volume_change = ((data.volume - data.avg_volume) /
                    data.avg_volume) * 100

    return {
        'avg_volume': float(data.avg_volume), # avg volume over last 30 days for APPL
        'latest_volume': int(data.volume),
        'volume_change_percent': float(volume_change)
    }

In [34]:
get_volume_trend(qs)

{'avg_volume': 1386.6,
 'latest_volume': 1874,
 'volume_change_percent': 35.15072840040388}

In [35]:
# Calculates a simplified price target based on past stock prices over a specified number of days or timestamps

def get_simple_target(ticker, timestamps=[], days=180):
    """
    Simplified price target calculation
    """
    end_date = timezone.now()
    start_date = end_date - timedelta(days=days)
    lookups = {
        "company__ticker": ticker,
        "time__range": (start_date, end_date)
    }
    if len(timestamps) > 0:
        lookups['time__in'] = timestamps
    daily_data = (
        StockQuote.timescale
        .filter(**lookups)
        .time_bucket('time', '1 day')
        .annotate(
            latest_price=Window(
                expression=FirstValue('close_price'),
                partition_by=[],
                order_by=F('time').desc()
            )
        )
        .aggregate(
            current_price=Max('latest_price'),
            avg_price=Avg('close_price'),
            highest=Max('high_price'),
            lowest=Min('low_price')
        )
    )
    
    if not daily_data:
        return None
        
    current_price = float(daily_data['current_price'])
    avg_price = float(daily_data['avg_price'])
    price_range = float(daily_data['highest']) - float(daily_data['lowest'])
    
    # Simple target based on average price and recent range
    conservative_target = current_price + (price_range * 0.382)  # 38.2% Fibonacci
    aggressive_target = current_price + (price_range * 0.618)   # 61.8% Fibonacci
    
    return {
        'current_price': current_price,
        'conservative_target': conservative_target,
        'aggressive_target': aggressive_target,
        'average_price': avg_price
    }

In [36]:
get_simple_target("AAPL")

{'current_price': 235.53,
 'conservative_target': 253.54512,
 'aggressive_target': 264.67488,
 'average_price': 221.40829961736645}

In [37]:
# get_simple_target("AAPL")